In [1]:
import os 
from urllib.parse import urlparse

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\surya\\CampusX\\End-to-end-Chest-Cancer-Prediction'

In [4]:
import dagshub
dagshub.init(repo_owner='SKC9372', repo_name='End-to-end-Chest-Cancer-Prediction', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

ModuleNotFoundError: No module named 'dagshub'

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/SKC9372/End-to-end-Chest-Cancer-Prediction.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'SKC9372'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '83e8cb7562ca5b943e963d9d311ed09a1c20f3fe'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model(r'artifacts\training\model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    test_data: Path
    all_params: Path
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [17]:
from src.CancerClassification.utils.common import create_directories,read_yaml,save_json
from src.CancerClassification.constants import  *

class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,params=PARAMS_FILE_PATH):
        self.config = read_yaml(config)
        self.params = read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model=r'artifacts\training\model.h5',
            test_data=r'artifacts\data_ingestion\Data\test',
            all_params=self.params,
            mlflow_uri='https://dagshub.com/SKC9372/End-to-end-Chest-Cancer-Prediction.mlflow',
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
            )
        return eval_config
    
    



In [15]:
import mlflow.keras


class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation='bilinear'
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.test_datagenerator = test_datagenerator.flow_from_directory(
            directory = Path(self.config.test_data),
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.test_datagenerator)
        self.save_score()
    
    def save_score(self):
        scores = {'loss':self.score[0],'accuracy':self.score[1]}
        save_json(path=Path(scores.json),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'loss':self.score[0],'accuracy':self.score[1]}
                               )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name='VGG16Model')
            else:
                mlflow.keras.log_model(self.model,"model")


    

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-04 15:44:51,156: INFO: common:  config\config.yaml:YAML file loaded successfully.]
[2025-01-04 15:44:51,163: INFO: common:  params.yaml:YAML file loaded successfully.]
[2025-01-04 15:44:51,167: INFO: common: Created directory: artifacts]
Found 315 images belonging to 4 classes.
20/20 [==============================] - 41s 2s/step - loss: 4.0121 - accuracy: 0.5048


2025/01/04 15:45:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-04 15:45:56,442: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\surya\AppData\Local\Temp\tmpun9hyyf2\model\data\model\assets
[2025-01-04 15:45:57,888: INFO: builder_impl: Assets written to: C:\Users\surya\AppData\Local\Temp\tmpun9hyyf2\model\data\model\assets]


c:\Users\surya\anaconda3\envs\chest_cancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


[2025-01-04 15:50:48,034: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /SKC9372/End-to-end-Chest-Cancer-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/cddb2b5362f04682a83e98dae37dcb0f/e5c3a562dbcc495fa3e585cb1ffb545a/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2025-01-04 15:51:26,039: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=4, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2426)')': /SKC9372/End-to-end-Chest-Cancer-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/cddb2b5362f04682a83e98dae37dcb0f/e5c3a562dbcc495fa3e585cb1ffb545a/artifacts/model/data/model/variables/variables.data-00000-of-00001]


Successfully registered model 'VGG16Model'.
2025/01/04 15:59:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
